In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings

In [2]:
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
def delete_duplicates(data,param_combined,param1,param2):
    data[param_combined] = data[param1].astype(str) + ' ' + data[param2].astype(str)
    data = data.drop_duplicates(subset=param_combined)
    data = data.drop(columns=param_combined)
    return data

In [4]:
bond_identity = pd.read_csv('bond_msbond.csv')
bond_price_1 = pd.read_csv('Bond_TrBei_to_20240816.csv')
bond_price_2 = pd.read_csv('Bond_TrBei_from_20240819_to_20250226.csv')
bond_price = pd.concat([bond_price_1,bond_price_2])
bond = bond_price.merge(bond_identity,on='portId',how='left')
bond['issuedyear'] = bond['issueddate'].str[:4]
bond['issuedyear'] = bond['issuedyear'].astype('int')
bond = bond.drop(columns='txtNo')
bond = delete_duplicates(data=bond,param_combined='portId + issueddate',param1='portId',param2='issueddate')

/var/folders/rj/b4yv8pv54v31_6jpqn2byxgc0000gn/T/ipykernel_83788/2532939075.py:2: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  bond_price_1 = pd.read_csv('Bond_TrBei_to_20240816.csv')


In [5]:
bond['portReleased'] = bond['portReleased'].replace(r'\N',np.nan)
bond['jtdatedt'] = pd.to_datetime(bond['jtdate'])
bond['portReleaseddt'] = pd.to_datetime(bond['portReleased'])
bond['tenorday'] = bond['jtdatedt'] - bond['portReleaseddt']
bond['tenorday'] = pd.to_numeric(bond['tenorday'].astype('str').str[:-5],errors='coerce')
bond['tenoryear'] = bond['tenorday']/365
bond = bond.drop(columns=['jtdatedt','portReleaseddt'])
bond = delete_duplicates(data=bond,param_combined='portId + issueddate',param1='portId',param2='issueddate')

In [6]:
cols = list(bond.columns)
bond['tradeday'] = 'Y'
bond_i_list = []
for i in bond['portId'].unique():
    bond_i = bond[bond['portId']==i]
    start_date = bond_i['portReleased'].unique()[0]
    end_date = max(bond['issueddate'].unique())
    jt_date = bond_i['jtdate'].unique()[0]
    if type(jt_date) is str:
        if end_date>jt_date:
            end_date = jt_date
        else:
            pass
    else:
        print(f'JT date for {i} is not found')
        continue
    if type(start_date) is str:
        bond_i_date = pd.DataFrame({'issueddate':pd.date_range(start=start_date,end=end_date,freq='D')})
        if len(bond_i_date)>0:
            bond_i_date['issueddate'] = bond_i_date['issueddate'].astype('str')
            bond_i = bond_i_date.merge(bond_i,on='issueddate',how='left')
            bond_i = bond_i.sort_values(by='issueddate')
            bond_i.loc[:,cols] = bond_i.loc[:,cols].ffill()
            bond_i_list.append(bond_i)
        else:
            bond_i_list.append(bond_i)
    else:
        print(f'Released date for {i} is not found')
        continue
bond = pd.concat(bond_i_list)
bond = bond.dropna(subset='portId')
bond = delete_duplicates(data=bond,param_combined='portId + issueddate',param1='portId',param2='issueddate')
bond = bond.reset_index(drop=True)
bond['tradeday'] = bond['tradeday'].fillna('N')
bond['weekend'] = np.where((pd.to_datetime(bond['issueddate'], errors='coerce')).dt.dayofweek<5,'N','Y')



Released date for BBNI01XXBFTW is not found
Released date for BBTN11 is not found
Released date for BEXI03C is not found
Released date for BFNC02 is not found
Released date for BJBR05 is not found
Released date for FR0021 is not found
Released date for FR0022 is not found
Released date for FR0023 is not found
Released date for FR0025 is not found
Released date for FR0026 is not found
Released date for FR0027 is not found
Released date for FR0033 is not found
Released date for FR0049 is not found
Released date for GBRB0017NvBF is not found
Released date for GBRB0019NvBF is not found
Released date for GBRB0020NvBF is not found
Released date for IFR0001 is not found
Released date for INDF04 is not found
Released date for ISAT04A is not found
Released date for ISAT06A is not found
Released date for ORI003 is not found
Released date for ORI004 is not found
Released date for ORI005 is not found
Released date for ORI006 is not found
Released date for OTMA06C is not found
Released date for OTM

In [7]:
#Tambah kurs BI
kurs_BI = pd.read_excel('kurs_BI.xls')
kurs_BI.columns = kurs_BI.loc[0]
kurs_BI = kurs_BI[1:].reset_index(drop=True)
kurs_BI = kurs_BI.rename(columns={'Nilai': 'USDIDR', 'Tanggal': 'issueddate'})
kurs_BI['issueddate'] = kurs_BI['issueddate'].apply(lambda x: x.strftime('%Y-%m-%d'))
bond = pd.merge(bond, kurs_BI[['issueddate', 'USDIDR']], on='issueddate', how='left')
bond_list = []
for i in bond['portId'].unique():
    bond_i = bond[bond['portId']==i]
    bond_i = bond_i.sort_values(by='issueddate')
    bond_i['USDIDR'] = bond_i['USDIDR'].ffill()
    bond_list.append(bond_i)
bond = pd.concat(bond_list)
bond = delete_duplicates(data=bond,param_combined='portId + issueddate',param1='portId',param2='issueddate')
bond = bond.reset_index(drop=True)
bond['outstanding'] = np.where(bond['dollar']=='Y',bond['outstanding']*bond['USDIDR'],bond['outstanding'])

In [8]:
#Tambah nominal
bond_nominal = pd.read_csv('bond_msnominal.csv').drop('txtno',axis=1).rename(columns={'portid': 'portId','tanggal': 'issueddate'})
bond_nominal['portId + issueddate'] = bond_nominal['portId'].astype(str) + ' ' + bond_nominal['issueddate'].astype(str)
bond['portId + issueddate'] = bond['portId'].astype(str) + ' ' + bond['issueddate'].astype(str)
bond = bond.merge(bond_nominal[['portId + issueddate','nominal']],how='left',on='portId + issueddate')
bond = bond.drop(columns='portId + issueddate')
bond_list = []
for i in bond['portId'].unique():
    bond_i = bond[bond['portId']==i]
    bond_i = bond_i.sort_values(by='issueddate')
    outstanding_i = bond_i['outstanding'].dropna().unique()[0]
    bond_i['nominal'] = bond_i['nominal'].ffill()
    bond_i['nominal'] = bond_i['nominal'].fillna(outstanding_i)
    bond_list.append(bond_i)
bond = pd.concat(bond_list)
bond = delete_duplicates(data=bond,param_combined='portId + issueddate',param1='portId',param2='issueddate')
bond = bond.reset_index(drop=True)



In [9]:
#Tambah accrued interest
bond_list = []
for i in bond['portId'].unique():
    bond_i = bond[bond['portId']==i]
    coupon_i = bond_i['coupon'].unique()[0]*0.01
    outstanding_i = bond_i['outstanding'].unique()[0]
    bond_i['interest'] = (coupon_i*outstanding_i)/365
    bond_i = bond_i.sort_values(by='issueddate')
    bond_i['accruedinterest'] = bond_i['interest'].cumsum()
    bond_list.append(bond_i)
bond = pd.concat(bond_list)
bond = delete_duplicates(data=bond,param_combined='portId + issueddate',param1='portId',param2='issueddate')
bond = bond.reset_index(drop=True)
bond['marketvalue'] = bond['nominal']*bond['lastPrice']*0.01 + bond['accruedinterest']


In [10]:
sbn = bond[bond['portType']=='S']
corpo = bond[bond['portType']=='B']

In [11]:
sbn_bday = sbn[sbn['weekend']=='N']

In [12]:
#Calculate average and sum transaction value over 3 months
period = 21*3 #days
date_list = [i.strftime('%Y-%m-%d') for i in pd.bdate_range(min(bond['issueddate']),max(bond['issueddate']))]
sbn_j_list = []
for en,i in enumerate(set(list(sbn_bday['portId']))):
    sbn_i = sbn_bday[sbn_bday['portId']==i].drop_duplicates()
    sbn_i = sbn_i.sort_values(by='issueddate')
    sbn_i['issueddatedt'] = pd.to_datetime(sbn_i['issueddate'])
    for j in list(sbn_i['issueddate'].unique()):
        sbn_j = sbn_i[sbn_i['issueddate']==j]
        try:
            pd_pos = date_list.index(j) - period #previous date index position
            cd_ = datetime.datetime.strptime(j, '%Y-%m-%d')
            pd_ = datetime.datetime.strptime(date_list[pd_pos], '%Y-%m-%d')
            sbn_i_inside_dates = sbn_i[(sbn_i['issueddatedt']>=pd_)&(sbn_i['issueddatedt']<=cd_)]
            #sbn_i_inside_dates = delete_duplicates(data=sbn_i_inside_dates,param_combined='portId + issueddate',param1='portId',param2='issueddate')
            if len(sbn_i_inside_dates)>0:
                sbn_j[f'sumvalueTrans_{period}'] = sum(sbn_i_inside_dates['valueTrans'])
                sbn_j[f'avgvalueTrans_{period}'] = sum(sbn_i_inside_dates['valueTrans'])/len(sbn_i_inside_dates['valueTrans'])
                sbn_j[f'pctdayvalueTrans_{period}'] = len(sbn_i_inside_dates['valueTrans'])/(period+1)
                sbn_j_list.append(sbn_j)
                if len(sbn_i_inside_dates['valueTrans'])>(period+1):
                    print(i)
                    print(j)
                else:
                    pass
            else: pass
        except ValueError: pass
sbn_vt = pd.concat(sbn_j_list)
sbn_vt = sbn_vt.reset_index(drop=True)
sbn_vt = sbn_vt.drop(columns='issueddatedt')


In [ ]:
#Create index
def create_bond_index(data,start_date,end_date,start_nab):
    date_list = [i.strftime('%Y-%m-%d') for i in pd.date_range(start_date,end_date)]
    data_list = []
    for en,i in enumerate(date_list):
        if en<1:
            data_i = data[data['issueddate']==i].drop_duplicates(subset='portId')
            data_i = data_i.dropna(subset='marketvalue')
            total_nominal= data_i['nominal'].sum()
            total_interest = data_i['interest'].sum()
            total_accruedinterest = data_i['accruedinterest'].sum()
            total_marketvalue = data_i['marketvalue'].sum()
            total_unit = total_marketvalue/start_nab
            nab = total_marketvalue/total_unit
            df = pd.DataFrame({'Date':i,'Day':[pd.to_datetime(i).day_name()],'Total Nominal':[total_nominal],'Total Interest':[total_interest],'Total Accrued Interest':[total_accruedinterest],
                            'Total Market Value':[total_marketvalue],'Total Unit':[total_unit],'NAB':[nab]},index=[en])
            data_list.append(df)
        else:
            data_i_t_1 = data_list[en-1]
            data_i = data[data['issueddate']==i].drop_duplicates(subset='portId')
            data_i = data_i.dropna(subset='marketvalue')
            total_nominal= data_i['nominal'].sum()
            total_interest = data_i['interest'].sum()
            total_accruedinterest = total_accruedinterest+total_interest
            total_marketvalue = data_i['marketvalue'].sum()
            nab = (total_marketvalue-(total_nominal-data_i_t_1['Total Nominal'].values[0]))/total_unit
            total_unit = total_unit+(total_nominal-data_i_t_1['Total Nominal'].values[0])/nab
            df = pd.DataFrame({'Date':i,'Day':[pd.to_datetime(i).day_name()],'Total Nominal':[total_nominal],'Total Interest':[total_interest],'Total Accrued Interest':[total_accruedinterest],
                            'Total Market Value':[total_marketvalue],'Total Unit':[total_unit],'NAB':[nab]},index=[en])
            data_list.append(df)
    bond_index = pd.concat(data_list)
    return bond_index

In [86]:
bond_index = create_bond_index(data=sbn,start_date='2022-02-22',end_date='2025-02-22',start_nab=10000)

In [ ]:
reb_date = ['2025-02-03','2024-10-01','2024-07-01','2024-04-01']

In [ ]:
# Market Value - Accrued Interest
# Pietro Veronesi - Fixed Income Securities_ Valuation, Risk, and Risk Management (2010, Wiley)
# Chapter 2.4, 2.4.4.2 Treasury Coupon Notes and Bonds page 51

with pd.ExcelWriter(f'obligasi_negara_tanpa_min_jatuh_tempo_0_MV_new.xlsx') as writer:
    for date_ in reb_date:
        sbn_1 = sbn_vt_df.copy()
        sbn_1['marketvalue'] = sbn_1['lastPrice']*sbn_1['outstanding adj'] # Dikurangin accrual
        sbn_1['date'] = date_
        sbn_1['date'] = pd.to_datetime(sbn_1['date'])
        sbn_1['jtdatedt'] = pd.to_datetime(sbn_1['jtdate'])
        sbn_1['issueddatedt'] = pd.to_datetime(sbn_1['issueddate'])
        sbn_1 = sbn_1[sbn_1['jtdatedt']>date_]
        sbn_1 = sbn_1[sbn_1['issueddatedt']<date_]
        cd_pos = date_list.index(date_)
        pd_pos = cd_pos - rolling_dic[period]
        cd_ = datetime.datetime.strptime(date_, '%Y-%m-%d')
        pd_ = datetime.datetime.strptime(date_list[pd_pos], '%Y-%m-%d')
        sbn_1 = sbn_1[(sbn_1['issueddatedt']>=pd_)&(sbn_1['issueddatedt']<=cd_)]
        
        # Market Value - Accrued Interest
        coupon = sbn_1['coupon'] / 100
        outstanding = sbn_1['outstanding adj']
        periodekupon = sbn_1['periodekupon']  # Periode Kupon per tahun
        tenorday = sbn_1['tenorday']  # Umur Obligasi
        jtdate = sbn_1['jtdatedt']  # Tanggal Maturity
        valid_condition = (tenorday > 0.0) & (periodekupon >= 1.0) & (tenorday < 1e10) & (periodekupon < 1e15) & (outstanding > 0.0)
        coupon_interval = tenorday / periodekupon
        coupon_interval[~valid_condition] = None  # Set NaN, -inf, inf to None
        last_coupon_date = jtdate - pd.to_timedelta(coupon_interval, unit='D')
        days_since_last_coupon = abs((pd.to_datetime('today') - last_coupon_date).dt.days)
        # print((coupon == 0).sum(), (outstanding == 0).sum(), (periodekupon == 0).sum(), (days_since_last_coupon == 0).sum(), (coupon_interval == 0).sum())
        sbn_1['accrued_interest'] = (coupon * outstanding / periodekupon) * (days_since_last_coupon / coupon_interval) # Calculate accrued interest
        sbn_1['marketvalue_accruedinterest'] = (sbn_1['lastPrice'] * sbn_1['outstanding adj']) - sbn_1['accrued_interest'] # Calculate market value

        sbn_1 = sbn_1[['portId', 'portName', 'date','issueddate',
                    'portReleased','jtdate','avgvalueTrans 3 Month','lastPrice',
                    'outstanding adj','marketvalue', 'marketvalue_accruedinterest', 'accrued_interest']]
        
        sbn_1 = sbn_1.drop_duplicates(subset='portId',keep='last')
        sbn_1 = sbn_1.dropna(subset='marketvalue')
        sbn_1 = sbn_1[sbn_1['marketvalue']!=0]
        sbn_1['MV Weight'] = sbn_1['marketvalue']/sum(sbn_1['marketvalue'])
        sbn_1 = sbn_1.dropna(subset='marketvalue_accruedinterest')
        sbn_1 = sbn_1[sbn_1['marketvalue_accruedinterest']!=0]
        sbn_1['MV - AI Weight'] = sbn_1['marketvalue_accruedinterest']/sum(sbn_1['marketvalue_accruedinterest'])
        sbn_1 = sbn_1.rename(columns={'portId':'ID','portName':'Nama Obligasi','date':'Rebalancing Date',
                                    'issueddate':'Last Transaction Date','portReleased':'Issuance Date',
                                    'jtdate':'Jatuh Tempo','outstanding adj':'Nilai Outstanding','avgvalueTrans 3 Month':'Average Transaction Value 3 Month',
                                    'lastPrice':'Last Price','marketvalue':'Market Value', 'marketvalue_accruedinterest': 'MV - AI', 'accrued_interest': 'AI'})
        sbn_1['Rebalancing Date'] = sbn_1['Rebalancing Date'].astype('str')
        sbn_1 = sbn_1.sort_values(by='Average Transaction Value 3 Month',ascending=False)
        # sbn_1 = sbn_1.head(50) #Ambil 50 bond
        sbn_1 = sbn_1.reset_index(drop=True)
        sbn_1.to_excel(writer, sheet_name=f'{date_}')

sbn_1

In [ ]:
# Market Value - Accrued Interest (Excel IGBI 2025)

with pd.ExcelWriter(f'obligasi_negara_tanpa_min_jatuh_tempo_0_MV_new.xlsx') as writer:
    for date_ in reb_date:
        sbn_1 = sbn_vt_df.copy()
        sbn_1['marketvalue'] = sbn_1['lastPrice']*sbn_1['outstanding adj'] # Dikurangin accrual
        sbn_1['date'] = date_
        sbn_1['date'] = pd.to_datetime(sbn_1['date'])
        sbn_1['jtdatedt'] = pd.to_datetime(sbn_1['jtdate'])
        sbn_1['issueddatedt'] = pd.to_datetime(sbn_1['issueddate'])
        sbn_1 = sbn_1[sbn_1['jtdatedt']>date_]
        sbn_1 = sbn_1[sbn_1['issueddatedt']<date_]
        cd_pos = date_list.index(date_)
        pd_pos = cd_pos - rolling_dic[period]
        cd_ = datetime.datetime.strptime(date_, '%Y-%m-%d')
        pd_ = datetime.datetime.strptime(date_list[pd_pos], '%Y-%m-%d')
        sbn_1 = sbn_1[(sbn_1['issueddatedt']>=pd_)&(sbn_1['issueddatedt']<=cd_)]
        
        # Market Value - Accrued Interest
        coupon = sbn_1['coupon']/100
        outstanding = sbn_1['outstanding adj']
        ytm = sbn_1['YTMValue']/100 + 1
        valid_condition = (coupon > 0.0) & (outstanding > 0.0) & (ytm > 0.0)
        condition_check = coupon * ytm * outstanding
        condition_check[~valid_condition] = None  # Set NaN, -inf, inf to None
        
        # print((coupon == 0).sum(), (outstanding == 0).sum(), (ytm == 0).sum())
        sbn_1['accrued_interest'] = coupon * ytm * outstanding / 365 # Calculate accrued interest
        sbn_1['marketvalue_accruedinterest'] = (sbn_1['lastPrice'] * sbn_1['outstanding adj']) - sbn_1['accrued_interest'] # Calculate market value

        sbn_1 = sbn_1[['portId', 'portName', 'date','issueddate',
                    'portReleased','jtdate','avgvalueTrans 3 Month','lastPrice',
                    'outstanding adj','marketvalue', 'marketvalue_accruedinterest', 'accrued_interest']]
        
        sbn_1 = sbn_1.drop_duplicates(subset='portId',keep='last')
        sbn_1 = sbn_1.dropna(subset='marketvalue')
        sbn_1 = sbn_1[sbn_1['marketvalue']!=0]
        sbn_1['MV Weight'] = sbn_1['marketvalue']/sum(sbn_1['marketvalue'])
        sbn_1 = sbn_1.dropna(subset='marketvalue_accruedinterest')
        sbn_1 = sbn_1[sbn_1['marketvalue_accruedinterest']!=0]
        sbn_1['MV - AI Weight'] = sbn_1['marketvalue_accruedinterest']/sum(sbn_1['marketvalue_accruedinterest'])
        sbn_1 = sbn_1.rename(columns={'portId':'ID','portName':'Nama Obligasi','date':'Rebalancing Date',
                                    'issueddate':'Last Transaction Date','portReleased':'Issuance Date',
                                    'jtdate':'Jatuh Tempo','outstanding adj':'Nilai Outstanding','avgvalueTrans 3 Month':'Average Transaction Value 3 Month',
                                    'lastPrice':'Last Price','marketvalue':'Market Value', 'marketvalue_accruedinterest': 'MV - AI', 'accrued_interest': 'AI'})
        sbn_1['Rebalancing Date'] = sbn_1['Rebalancing Date'].astype('str')
        sbn_1 = sbn_1.sort_values(by='Average Transaction Value 3 Month',ascending=False)
        # sbn_1 = sbn_1.head(50) #Ambil 50 bond
        sbn_1 = sbn_1.reset_index(drop=True)
        sbn_1.to_excel(writer, sheet_name=f'{date_}')

sbn_1

In [ ]:
with pd.ExcelWriter(f'obligasi_negara_tanpa_min_jatuh_tempo_0.xlsx') as writer:
    for date_ in reb_date:
        sbn_1 = sbn_vt.copy()
        sbn_1['marketvalue'] = sbn_1['lastPrice']*sbn_1['outstanding'] # Dikurangin accrual
        # sbn_1['accruaedinterest'] = 
        sbn_1['date'] = date_
        sbn_1['date'] = pd.to_datetime(sbn_1['date'])
        sbn_1['jtdatedt'] = pd.to_datetime(sbn_1['jtdate'])
        sbn_1['issueddatedt'] = pd.to_datetime(sbn_1['issueddate'])
        sbn_1 = sbn_1[sbn_1['jtdatedt']>date_]
        sbn_1 = sbn_1[sbn_1['issueddatedt']<date_]
        cd_pos = date_list.index(date_)
        pd_pos = cd_pos - rolling_dic[period]
        cd_ = datetime.datetime.strptime(date_, '%Y-%m-%d')
        pd_ = datetime.datetime.strptime(date_list[pd_pos], '%Y-%m-%d')
        sbn_1 = sbn_1[(sbn_1['issueddatedt']>=pd_)&(sbn_1['issueddatedt']<=cd_)]
        sbn_1 = sbn_1[['portId','portName','date','issueddate','portReleased','jtdate','avgvalueTrans 3 Month','lastPrice','outstanding','marketvalue']]
        sbn_1 = sbn_1.drop_duplicates(subset='portId',keep='last')
        sbn_1 = sbn_1.dropna(subset='marketvalue')
        sbn_1 = sbn_1[sbn_1['marketvalue']!=0]
        sbn_1['Weight'] = sbn_1['marketvalue']/sum(sbn_1['marketvalue'])
        sbn_1 = sbn_1.rename(columns={'portId':'ID','portName':'Nama Obligasi','date':'Rebalancing Date','issueddate':'Last Transaction Date','portReleased':'Issuance Date','jtdate':'Jatuh Tempo','outstanding':'Nilai Outstanding','avgvalueTrans 3 Month':'Average Transaction Value 3 Month','lastPrice':'Last Price','marketvalue':'Market Value'})
        sbn_1['Rebalancing Date'] = sbn_1['Rebalancing Date'].astype('str')
        sbn_1 = sbn_1.sort_values(by='Average Transaction Value 3 Month',ascending=False)
        sbn_1 = sbn_1.head(50) #Ambil 50 bond
        sbn_1 = sbn_1.reset_index(drop=True)
        sbn_1.to_excel(writer, sheet_name=f'{date_}')